<a href="https://colab.research.google.com/github/Anu0603/Anu0603/blob/main/ML_Model_Building_and_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/laxmimerit/preprocess_kgptalkie.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/laxmimerit/preprocess_kgptalkie.git to /tmp/pip-req-build-xg7iyh9o
  Running command git clone -q https://github.com/laxmimerit/preprocess_kgptalkie.git /tmp/pip-req-build-xg7iyh9o
  Created wheel for preprocess-kgptalkie: filename=preprocess_kgptalkie-0.1.3-py3-none-any.whl size=11756 sha256=fb729712c3480cc3e8552419af0890c89a5324b8ba2cad249399e9ac665ab18e
  Stored in directory: /tmp/pip-ephem-wheel-cache-_0fya92i/wheels/0d/b3/29/bfe3deffda68980088d17b81331be6667e837ffb4a071bae82
Successfully built preprocess-kgptalkie


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix




In [ ]:
!pip install spacy==2.2.3
!python -m spacy download en_core_web_sm
!pip install beautifulsoup4==4.9.1
!pip install textblob==0.15.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 10.4 MB 5.1 MB/s 
     |████████████████████████████████| 2.2 MB 38.7 MB/s 
     |████████████████████████████████| 3.7 MB 27.8 MB/s 
     |████████████████████████████████| 184 kB 43.3 MB/s 
  Attempting uninstall: srsly
    Found existing installation: srsly 2.4.4
    Uninstalling srsly-2.4.4:
      Successfully uninstalled srsly-2.4.4
  Attempting uninstall: blis
    Found existing installation: blis 0.7.8
    Uninstalling blis-0.7.8:
      Successfully uninstalled blis-0.7.8
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.0
    Uninstalling thinc-8.1.0:
      Successfully uninstalled thinc-8.1.0
  Attempting uninstall: catalogue
    Found existing installation: catalogue 2.0.8
    Uninstalling catalogue-2.0.8:
      Successfully uninstalled catalogue-2.0.8
  Attempting uninstall: spacy
    Found existing installation: sp

In [ ]:
import preprocess_kgptalkie as ps

In [ ]:

#sample code
x = "A very, very, very slow-\@#moving, aimless** movie ankita@gmail.com"
ps.remove_special_chars(x)
ps.remove_accented_chars(x)
ps.cont_exp(x)
ps.remove_emails(x)
ps.remove_html_tags(x)
ps.remove_urls(x)



'A very, very, very slow-\\@#moving, aimless** movie ankita@gmail.com'

In [ ]:
df = pd.read_csv('imdb_reviews.txt', sep = '\t', header = None)

In [ ]:
df.columns = ['reviews', 'sentiment']

In [ ]:
df.head()

,reviews,sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [ ]:
df['reviews'] = df['reviews'].apply(lambda x: ps.cont_exp(x))
df['reviews'] = df['reviews'].apply(lambda x: ps.remove_special_chars(x))
df['reviews'] = df['reviews'].apply(lambda x: ps.remove_accented_chars(x))
df['reviews'] = df['reviews'].apply(lambda x: ps.remove_emails(x))
df['reviews'] = df['reviews'].apply(lambda x: ps.remove_html_tags(x))
df['reviews'] = df['reviews'].apply(lambda x: ps.remove_urls(x))
df['reviews'] = df['reviews'].apply(lambda x: ps.make_base(x))
df['reviews'] = df['reviews'].apply(lambda x: str(x).lower())


In [ ]:
df.head()

,reviews,sentiment
0,a very very very slowmove aimless movie about ...,0
1,not sure who was more lose the flat character ...,0
2,attempt artiness with black white and clever c...,0
3,very little music or anything to speak of,0
4,the good scene in the movie was when gerardo i...,1


## Data Preparation for Model Training

# New Section

In [ ]:
X = df['reviews']
y = df['sentiment']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0, stratify = y)

X_train.shape, X_test.shape

((598,), (150,))

## ML Model Building 




In [ ]:
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression(solver = 'liblinear'))
])

In [ ]:
hyperparameters = {
    'tfidf__max_df': (0.5, 1.0),
    'tfidf__ngram_range': ((1,1), (1,2)),
    'tfidf__use_idf': (True, False),
    'tfidf__analyzer': ('word', 'char', 'char_wb'),
    'clf__penalty': ('l2', 'l1'),
    'clf__C': (1,2)
}

In [ ]:
clf=GridSearchCV(pipe,hyperparameters, n_jobs=None, cv=None)

In [ ]:
%%time
clf.fit(X_train, y_train)

CPU times: user 33.7 s, sys: 192 ms, total: 33.9 s
Wall time: 40.8 s


GridSearchCV(estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('clf',
                                        LogisticRegression(solver='liblinear'))]),
             param_grid={'clf__C': (1, 2), 'clf__penalty': ('l2', 'l1'),
                         'tfidf__analyzer': ('word', 'char', 'char_wb'),
                         'tfidf__max_df': (0.5, 1.0),
                         'tfidf__ngram_range': ((1, 1), (1, 2)),
                         'tfidf__use_idf': (True, False)})

In [ ]:
clf.best_estimator_

Pipeline(steps=[('tfidf', TfidfVectorizer()),
                ('clf', LogisticRegression(C=2, solver='liblinear'))])

In [ ]:
clf.best_params_

{'clf__C': 2,
 'clf__penalty': 'l2',
 'tfidf__analyzer': 'word',
 'tfidf__max_df': 1.0,
 'tfidf__ngram_range': (1, 1),
 'tfidf__use_idf': True}

In [ ]:
clf.best_score_

0.7758263305322128

In [ ]:
y_pred = clf.predict(X_test)


In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.70      0.70      0.70        73
           1       0.71      0.71      0.71        77

    accuracy                           0.71       150
   macro avg       0.71      0.71      0.71       150
weighted avg       0.71      0.71      0.71       150



In [ ]:
## SVM Model 


In [ ]:
from sklearn.svm import LinearSVC

In [ ]:
pipe=Pipeline([
    ('tfidf',TfidfVectorizer()),
    ('clf', LinearSVC())
])

In [ ]:
hyperparameters={
   'tfidf__max_df': (0.5, 1.0),
    'tfidf__ngram_range': ((1,1), (1,2)),
    'tfidf__use_idf': (True, False),
    'tfidf__analyzer': ('word', 'char', 'char_wb'),
    'clf__C': (1,2,2.5,3)
}

In [ ]:
clf=GridSearchCV(pipe, hyperparameters, n_jobs=-1, cv=5)

In [ ]:
%%time
clf.fit(X_train, y_train)

CPU times: user 1.75 s, sys: 48.3 ms, total: 1.8 s
Wall time: 28.1 s


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('tfidf', TfidfVectorizer()),
                                       ('clf', LinearSVC())]),
             n_jobs=-1,
             param_grid={'clf__C': (1, 2, 2.5, 3),
                         'tfidf__analyzer': ('word', 'char', 'char_wb'),
                         'tfidf__max_df': (0.5, 1.0),
                         'tfidf__ngram_range': ((1, 1), (1, 2)),
                         'tfidf__use_idf': (True, False)})

In [ ]:
clf.best_estimator_

In [ ]:
clf.best_params_

{'clf__C': 1,
 'tfidf__analyzer': 'word',
 'tfidf__max_df': 1.0,
 'tfidf__ngram_range': (1, 1),
 'tfidf__use_idf': True}

In [ ]:
clf.best_score_

0.7875770308123249

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.70      0.71      0.71        73
           1       0.72      0.71      0.72        77

    accuracy                           0.71       150
   macro avg       0.71      0.71      0.71       150
weighted avg       0.71      0.71      0.71       150



In [ ]:
## Model Testing and Saving 

In [ ]:
x = ['this is great moview. i loved it', 'i have watched this movie. plot is straight. return my money']

In [ ]:
clf.predict(x)

array([1, 0])

In [ ]:
#first one is [ositive and second one is negative]

In [ ]:
import pickle as pkl

In [ ]:
pkl.dump(clf,open('model.pkl','wb'))